## Gradient-weighted Regression Activation Maps (Grad-RAM)

In [83]:
from __future__ import print_function
import os
import numpy as np
import sys
import cv2
import scipy.io as sio
import matplotlib.pyplot as plt
import keras
import tensorflow as tf
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Lambda, Concatenate, Add, Maximum, Average
from keras.layers import Conv2D, Input, MaxPooling2D, AveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.models import Model
from keras.layers.core import Lambda
from keras.preprocessing import image

In [84]:
batch_size = 128
epochs = 10

## Load Data

In [85]:
dir_name = 'processed_data/expressive_data/'

density = '08'

# load train data

fname = 'x_train_expressive_density_' + density + '.mat'
filename = os.path.join(os.getcwd(),dir_name,fname)
x_train_mat = sio.loadmat(filename)['x_train_expressive_mat']
x_train_mat = np.float32(x_train_mat)

fname = 'y_train_expressive_density_' + density + '.mat'
filename = os.path.join(os.getcwd(),dir_name,fname)
y_train_mat = sio.loadmat(filename)['y_train_expressive_mat']
y_train_mat = np.float32(y_train_mat)

# load test data
fname = 'x_test_expressive_density_' + density + '.mat'
filename = os.path.join(os.getcwd(),dir_name,fname)
x_test_mat = sio.loadmat(filename)['x_test_expressive_mat']
x_test_mat = np.float32(x_test_mat)

fname = 'y_test_expressive_density_' + density + '.mat'
filename = os.path.join(os.getcwd(),dir_name,fname)
y_test_mat = sio.loadmat(filename)['y_test_expressive_mat']
y_test_mat = np.float32(y_test_mat)

print('x_train_mat_shape:',x_train_mat.shape)
print('x_test_mat_shape:',x_test_mat.shape)
print('y_train_mat_shape:',y_train_mat.shape)
print('y_test_mat_shape:',y_test_mat.shape)


# Form training and testing data

x_train = np.zeros((x_train_mat.shape[2],116,116,1),dtype=np.float32)
y_train = np.zeros((x_train_mat.shape[2],1),dtype=np.float32)

x_test = np.zeros((x_test_mat.shape[2],116,116,1),dtype=np.float32)
y_test = np.zeros((x_test_mat.shape[2],1),dtype=np.float32)


for i in range(x_train_mat.shape[2]):
    x_train[i,:,:,0] = x_train_mat[:,:,i]
    y_train[i,0] = y_train_mat[i,0] # EXPRESSIVE SCORE
        
for i in range(x_test_mat.shape[2]):
    x_test[i,:,:,0] = x_test_mat[:,:,i]
    y_test[i,0] = y_test_mat[i,0]  # EXPRESSIVE SCORE
    
print('x_train_shape:',x_train.shape)
print('x_test_shape:',x_test.shape)
print('y_train_shape:',y_train.shape)
print('y_test_shape:',y_test.shape)

x_train_mat_shape: (116, 116, 17374)
x_test_mat_shape: (116, 116, 8687)
y_train_mat_shape: (17374, 1)
y_test_mat_shape: (8687, 1)
x_train_shape: (17374, 116, 116, 1)
x_test_shape: (8687, 116, 116, 1)
y_train_shape: (17374, 1)
y_test_shape: (8687, 1)


In [86]:
# Reshape input
print(x_train.shape)
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 1)
input_shape = (x_train.shape[1], x_train.shape[2], 1)

print('x_train_shape:',x_train.shape)
print('x_test_shape:',x_test.shape)
print('input_shape:',input_shape)


(17374, 116, 116, 1)
x_train_shape: (17374, 116, 116, 1)
x_test_shape: (8687, 116, 116, 1)
input_shape: (116, 116, 1)


## Define Dilated CNN + RN Model

In [87]:
## Define dilated CNN
d_rate = 2 # dilation rate

# Define 4 convolutional layers
def ConvolutionNetworks(no_filters=32, kernel_size=3, stride_size=1):
    def conv(model):
        model = Conv2D(no_filters, (3,3), strides=(stride_size,stride_size), activation='relu', input_shape=input_shape, data_format='channels_last')(model)
        model = MaxPooling2D()(model)
        model = BatchNormalization()(model)
        
        model = Conv2D(no_filters, (3,3), strides=(stride_size,stride_size), activation='relu')(model)
        model = MaxPooling2D()(model)
        model = BatchNormalization()(model)
        
        model = Conv2D(no_filters, (3,3), strides=(stride_size,stride_size), dilation_rate=(d_rate,d_rate), activation='relu')(model)
        model = MaxPooling2D()(model)
        model = BatchNormalization()(model)
        
        model = Conv2D(no_filters, (3,3), strides=(stride_size,stride_size), dilation_rate=(d_rate,d_rate), activation='relu')(model) # d=3
        #model = Conv2D(16, (3,3), strides=(stride_size,stride_size), activation='relu')(model) # d=4
        model = MaxPooling2D()(model)
        model = BatchNormalization()(model)
        
        return model
    return conv

In [88]:
# Define function to compute relations from objects - the following uses just 4 Lambda layers - O(n^2) time complexity

def compute_relations(objects):
    
    def get_top_dim_1(t):
        return t[:,0,:,:]
    
    def get_all_but_top_dim_1(t):
        return t[:,1:,:,:]
    
    def get_top_dim_2(t):
        return t[:,0,:]
    
    def get_all_but_top_dim_2(t):
        return t[:,1:,:]
    
    slice_top_dim_1 = Lambda(get_top_dim_1)
    slice_all_but_top_dim_1 = Lambda(get_all_but_top_dim_1)
    slice_top_dim_2 = Lambda(get_top_dim_2)
    slice_all_but_top_dim_2 = Lambda(get_all_but_top_dim_2)
    
    d = K.int_shape(objects)[2]
    print('d = ', d)
    features = []
    
    for i in range(d):
        features1 = slice_top_dim_1(objects)
        objects = slice_all_but_top_dim_1(objects)
        
        for j in range(d):
            features2 = slice_top_dim_2(features1)
            features1 = slice_all_but_top_dim_2(features1)
            features.append(features2)
            
    relations = []
    concat = Concatenate()
    for feature1 in features:
        for feature2 in features:
            if (features.index(feature1) < features.index(feature2)): # remove symmetric and self-relations
                relations.append(concat([feature1,feature2]))
                    
            
    return relations

In [89]:
# Baseline model - f(theta) of RN
def f_theta():
    def f(model):
        model = Dense(512)(model)
        model = Activation('relu')(model)
        model = Dropout(0.5)(model)
        
        model = Dense(512)(model)
        model = Activation('relu')(model)
        model = Dropout(0.5)(model)
        
        model = Dense(512)(model)
        model = Activation('relu')(model)
        
        #model = Dense(100)(model)
        #model = Activation('relu')(model)
        
        return model
    return f

In [90]:
def g_th(layers):
    def f(model):
        for n in range(len(layers)):
            model = layers[n](model)
        return model
    return f

def stack_layer(layers):
    def f(x):
        for k in range(len(layers)):
            x = layers[k](x)
        return x
    return f

# Define g(theta) of RN model
def g_theta(units=512, layers=4):
    r = []
    for k in range(layers):
        r.append(Dense(units))
        r.append(Activation('relu'))
    return g_th(r)

def get_MLP():
    return g_th()

In [91]:
# Define the main RN
    
def RelationNetworks(objects):
    g_t = g_theta()
    relations = compute_relations(objects)
    print('No of Relations:', len(relations))
    #print(relations)
    
    g_all = []
    
    for i, r in enumerate(relations):
        g_all.append(g_t(r))
        
    # combine to make the network combinatorially generalizable
    #combined_relation = Add()(g_all)
    combined_relation = Maximum()(g_all) # max pooling gave best results
    #combined_relation = Average()(g_all)
    
    f_out = f_theta()(combined_relation)
    return f_out

In [92]:
# build tag to identify objects in the image
def build_tag(conv):
    d = K.int_shape(conv)[2]
    tag = np.zeros((d,d,2))
    
    for i in range(d):
        for j in range(d):
            tag[i,j,0] = float(int(i%d))/(d-1)*2-1
            tag[i,j,1] = float(int(j%d))/(d-1)*2-1
            
    tag = K.variable(tag)
    tag = K.expand_dims(tag, axis=0)
    batch_size = K.shape(conv)[0]
    tag = K.tile(tag, [batch_size,1,1,1])
    
    return Input(tensor=tag)

In [95]:
# Construct the model
input_img = Input((x_train.shape[1], x_train.shape[2], 1))  

print('Input_image',input_img.shape)

img_after_conv = ConvolutionNetworks()(input_img) 
#tag = build_tag(img_after_conv)
#img_after_conv = Concatenate()([tag, img_after_conv])

img_after_RN = RelationNetworks(img_after_conv)

img_out = Dense(1, activation='linear')(img_after_RN)

#RN_model = Model(inputs=[input_img, tag], outputs=img_out)

RN_model = Model(inputs=[input_img], outputs=img_out)

print(RN_model.summary())

Input_image (?, 116, 116, 1)
d =  3
No of Relations: 36
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_15 (InputLayer)           (None, 116, 116, 1)  0                                            
__________________________________________________________________________________________________
conv2d_29 (Conv2D)              (None, 114, 114, 32) 320         input_15[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_29 (MaxPooling2D) (None, 57, 57, 32)   0           conv2d_29[0][0]                  
__________________________________________________________________________________________________
batch_normalization_29 (BatchNo (None, 57, 57, 32)   128         max_pooling2d_29[0][0]           
_____________________________________________________

## Compile and Test Model

In [96]:
#compile model
adam = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

RN_model.compile(optimizer=adam, loss='mse', metrics=['mae','mape'])


# Train Model
RN_model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs)

# Save the trained Model
RN_model.save_weights('RN_model_weights.h5')

with open('RN_model_architecture.json','w') as f:
    f.write(RN_model.to_json())

# Test Model
y_test_pred = RN_model.predict(x_test)

print(y_test.shape)
print('y_test:', y_test)


print(y_test_pred.shape)
print('y_test_pred:', y_test_pred)


# Print Results
print('mae: ', np.mean(np.abs(y_test - y_test_pred)))
print('sdae: ', np.std(np.abs(y_test - y_test_pred)))

count = 0
for i in range(len(y_test)):
    if (np.abs(y_test[i] - y_test_pred[i])) < 0.125: # prob of mae less than 15 - 15/120 = 0.125
        count += 1
        
prob_error = count / len(y_test)

print('prob of mae less than 0.125: ', prob_error)

Epoch 1/10
17374/17374 [==============================] - 17s 959us/step - loss: 0.0534 - mean_absolute_error: 0.1813 - mean_absolute_percentage_error: 29.2885
Epoch 2/10
17374/17374 [==============================] - 9s 491us/step - loss: 0.0170 - mean_absolute_error: 0.1013 - mean_absolute_percentage_error: 15.9951
Epoch 3/10
17374/17374 [==============================] - 8s 489us/step - loss: 0.0090 - mean_absolute_error: 0.0732 - mean_absolute_percentage_error: 11.2984
Epoch 4/10
17374/17374 [==============================] - 8s 488us/step - loss: 0.0062 - mean_absolute_error: 0.0611 - mean_absolute_percentage_error: 9.2508
Epoch 5/10
17374/17374 [==============================] - 9s 492us/step - loss: 0.0057 - mean_absolute_error: 0.0584 - mean_absolute_percentage_error: 8.8517
Epoch 6/10
17374/17374 [==============================] - 8s 488us/step - loss: 0.0048 - mean_absolute_error: 0.0534 - mean_absolute_percentage_error: 8.0206
Epoch 7/10
17374/17374 [========================

In [97]:
import json
from keras.models import model_from_json, load_model

print('x_test_shape:',x_test.shape)

#x_test = x_test.reshape(x_test.shape[1], x_test.shape[2], x_test.shape[0])

#print('x_test_shape:',x_test.shape)

print(type(x_test.dtype))

x_test = np.asarray(x_test)
print('x_test_shape:',x_test.shape)
print(type(x_test.dtype))

x_test_img = x_test[0,:,:,:]
print('x_test_img_shape:',x_test_img.shape)

x_test_shape: (8687, 116, 116, 1)
<class 'numpy.dtype'>
x_test_shape: (8687, 116, 116, 1)
<class 'numpy.dtype'>
x_test_img_shape: (116, 116, 1)


In [98]:
from keras.preprocessing import image

In [101]:
## Load pretrained Model
with open('RN_model_architecture.json', 'r') as f:
    RN_model = model_from_json(f.read())

RN_model.load_weights('RN_model_weights.h5')

#compile model
adam = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

RN_model.compile(optimizer=adam, loss='mse', metrics=['mae','mape'])


#test_img = image.img_to_array(x_test[:,:,0])
#print('test_img_shape:',test_img.shape)
#print(type(test_img.dtype))


#test_img = np.expand_dims(test_img, axis=0)

# Test Model
y_test_pred = RN_model.predict(x_test)

print(y_test_pred)

[[0.40307313]
 [0.5544771 ]
 [0.58039826]
 ...
 [0.4247951 ]
 [0.46159256]
 [0.4210567 ]]


SyntaxError: invalid syntax (<ipython-input-50-318110c80eec>, line 5)